In [141]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score,accuracy_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from tensorflow.keras import layers


In [142]:
df=pd.read_csv("merged_student_data.csv")
del df[df.columns[0]]


In [143]:
df['Avg_x']=(df['G1_x']+df['G2_x']+df['G3_x'])/3
df['Avg_y']=(df['G1_y']+df['G2_y']+df['G3_y'])/3
df['Avg']=(df['Avg_x']+df['Avg_y'])/2


In [144]:
sex={
    'M':1,
    'F':0,
}
df['sex']=df['sex'].map(sex) 
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,Dalc_y,Walc_y,health_y,absences_y,G1_y,G2_y,G3_y,Avg_x,Avg_y,Avg
0,GP,0,18,U,GT3,A,4,4,at_home,teacher,...,1,1,3,4,0,11,11,5.666667,7.333333,6.500000
1,GP,0,17,U,GT3,T,1,1,at_home,other,...,1,1,3,2,9,11,11,5.333333,10.333333,7.833333
2,GP,0,15,U,LE3,T,1,1,at_home,other,...,2,3,3,6,12,13,12,8.333333,12.333333,10.333333
3,GP,0,15,U,GT3,T,4,2,health,services,...,1,1,5,0,14,14,14,14.666667,14.000000,14.333333
4,GP,0,16,U,GT3,T,3,3,other,other,...,1,2,5,0,11,13,13,8.666667,12.333333,10.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,MS,0,18,U,LE3,T,3,1,teacher,services,...,1,1,1,4,15,15,16,8.000000,15.333333,11.666667
378,MS,0,18,U,GT3,T,1,1,other,other,...,2,2,5,3,7,8,7,3.666667,7.333333,5.500000
379,MS,0,18,U,GT3,T,1,1,other,other,...,1,1,5,6,11,12,9,3.666667,10.666667,7.166667
380,MS,1,17,U,LE3,T,3,1,services,services,...,3,4,2,6,10,10,10,15.333333,10.000000,12.666667


In [145]:
# col=df.select_dtypes(include='object').columns

# for i in col:
#     print(i)
#     print(df[i].unique())
df.dtypes


school           object
sex               int64
age               int64
address          object
famsize          object
Pstatus          object
Medu              int64
Fedu              int64
Mjob             object
Fjob             object
reason           object
guardian_x       object
traveltime_x      int64
studytime_x       int64
failures_x        int64
schoolsup_x      object
famsup_x         object
paid_x           object
activities_x     object
nursery          object
higher_x         object
internet         object
romantic_x       object
famrel_x          int64
freetime_x        int64
goout_x           int64
Dalc_x            int64
Walc_x            int64
health_x          int64
absences_x        int64
G1_x              int64
G2_x              int64
G3_x              int64
guardian_y       object
traveltime_y      int64
studytime_y       int64
failures_y        int64
schoolsup_y      object
famsup_y         object
paid_y           object
activities_y     object
higher_y        

In [146]:
col=df.select_dtypes(include='int').columns

for i in col:
    print(i)
    print(df[i].unique())


sex
[0 1]
age
[18 17 15 16 19 22 20]
Medu
[4 1 3 2 0]
Fedu
[4 1 2 3 0]
traveltime_x
[2 1 3 4]
studytime_x
[2 3 1 4]
failures_x
[0 3 2 1]
famrel_x
[4 5 3 1 2]
freetime_x
[3 2 4 1 5]
goout_x
[4 3 2 1 5]
Dalc_x
[1 2 5 3 4]
Walc_x
[1 3 2 4 5]
health_x
[3 5 1 2 4]
absences_x
[ 6  4 10  2  0 16 14  7  8 25 12 54 18 26 20 56 24 28  5 13 15 22  3 21
  1 75 30 19  9 11 23 17]
G1_x
[ 5  7 15  6 12 16 14 10 13  8 11  9 17 19 18  4  3]
G2_x
[ 6  5  8 14 10 15 12 18 16 13  9 11  7 19  0 17]
G3_x
[ 6 10 15 11 19  9 12 14 16  5  8 17 18 13 20  7  0  4]
traveltime_y
[2 1 3 4]
studytime_y
[2 3 1 4]
failures_y
[0 3 1 2]
famrel_y
[4 5 3 1 2]
freetime_y
[3 2 4 1 5]
goout_y
[4 3 2 1 5]
Dalc_y
[1 2 5 3 4]
Walc_y
[1 3 2 4 5]
health_y
[3 5 1 2 4]
absences_y
[ 4  2  6  0 10  8 16 14  1 22 12 32 30 21 15  9  7 18  3]
G1_y
[ 0  9 12 14 11 13 10 15 17  8 16 18  7 19  5]
G2_y
[11 13 14 12 16 17  8 10 15  9  7 18 19  5]
G3_y
[11 12 14 13 17 15  7 10 16  9  6  8 18  0  1  5 19]


In [147]:
df=pd.get_dummies(df,drop_first=True)
df

,sex,age,Medu,Fedu,traveltime_x,studytime_x,failures_x,famrel_x,freetime_x,goout_x,...,internet_yes,romantic_x_yes,guardian_y_mother,guardian_y_other,schoolsup_y_yes,famsup_y_yes,paid_y_yes,activities_y_yes,higher_y_yes,romantic_y_yes
0,0,18,4,4,2,2,0,4,3,4,...,False,False,True,False,True,False,False,False,True,False
1,0,17,1,1,1,2,0,5,3,3,...,True,False,False,False,False,True,False,False,True,False
2,0,15,1,1,1,2,3,4,3,2,...,True,False,True,False,True,False,False,False,True,False
3,0,15,4,2,1,3,0,3,2,2,...,True,True,True,False,False,True,False,True,True,True
4,0,16,3,3,1,2,0,4,3,2,...,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0,18,3,1,1,2,0,4,3,4,...,True,False,True,False,False,True,False,False,True,False
378,0,18,1,1,2,2,1,1,1,1,...,False,False,True,False,False,False,False,True,True,True
379,0,18,1,1,2,2,1,1,1,1,...,False,False,True,False,False,False,False,True,True,False
380,1,17,3,1,2,1,0,2,4,5,...,True,False,True,False,False,False,False,False,True,False


In [148]:
x=df.drop(columns='Avg').values
x

array([[0, 18, 4, ..., False, True, False],
       [0, 17, 1, ..., False, True, False],
       [0, 15, 1, ..., False, True, False],
       ...,
       [0, 18, 1, ..., True, True, False],
       [1, 17, 3, ..., False, True, False],
       [1, 18, 3, ..., False, True, False]], dtype=object)

In [149]:
y=df['Avg'].values
y


array([ 6.5       ,  7.83333333, 10.33333333, 14.33333333, 10.5       ,
       13.66666667, 12.16666667,  8.83333333, 16.83333333, 13.5       ,
       11.5       , 11.5       , 13.16666667, 11.33333333, 14.83333333,
       15.5       , 13.5       , 11.5       ,  6.5       , 10.66666667,
       13.5       , 12.83333333, 14.16666667, 11.33333333,  9.66666667,
        9.33333333, 11.66666667, 13.16666667, 11.66666667, 11.33333333,
       10.66666667, 15.83333333, 15.16666667, 11.16666667, 12.83333333,
        9.        , 15.16666667, 14.16666667, 11.66666667, 13.16666667,
       10.        , 11.33333333, 16.5       ,  9.33333333, 10.16666667,
        9.        , 12.        , 18.16666667, 13.33333333,  9.66666667,
       13.        , 13.83333333, 10.        , 11.        , 12.33333333,
       10.66666667, 14.66666667, 15.        , 11.5       , 15.66666667,
       13.5       , 10.83333333, 10.5       , 11.33333333, 11.16666667,
       15.5       , 12.16666667,  8.16666667,  9.5       , 15.5 

In [150]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

RandomForestRegressor

In [151]:
# Create a Random Forest Regressor model
regressor = RandomForestRegressor()

# Fit the model to the training data
regressor.fit(x_train, y_train)

# Make predictions on the test data
y_pred = regressor.predict(x_test)

# Calculate the Mean Squared Error (MSE) to evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.16905028935185187


svm

In [152]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

regressor = SVR(kernel='linear')  
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.0031515712114867024


k-nearest neighbour


In [153]:

k = 5
knn_regressor = KNeighborsRegressor(n_neighbors=k)
knn_regressor.fit(x_train, y_train)
y_pred = knn_regressor.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.38121527777777775


Neural network

In [154]:
# Standardize the data (recommended for neural networks)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)


model = keras.Sequential()
model.add(layers.Input(shape=(x_train.shape[1],))) 
model.add(layers.Dense(64, activation='relu')) 
model.add(layers.Dense(32, activation='relu'))  
model.add(layers.Dense(1, activation='linear'))  


model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)


y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Epoch 1/100
9/9 [==============================] - 1s 13ms/step - loss: 133.7346 - val_loss: 121.4692
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 114.5210 - val_loss: 103.8627
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 96.7049 - val_loss: 86.3854
Epoch 4/100
9/9 [==============================] - 0s 5ms/step - loss: 78.6676 - val_loss: 68.1026
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 59.6139 - val_loss: 49.4168
Epoch 6/100
9/9 [==============================] - 0s 4ms/step - loss: 40.3329 - val_loss: 31.9203
Epoch 7/100
9/9 [==============================] - 0s 6ms/step - loss: 23.6735 - val_loss: 17.7450
Epoch 8/100
9/9 [==============================] - 0s 6ms/step - loss: 11.7648 - val_loss: 9.4646
Epoch 9/100
9/9 [==============================] - 0s 6ms/step - loss: 6.3905 - val_loss: 6.7886
Epoch 10/100
9/9 [==============================] - 0s 5ms/step - loss: 5.0446 - val_loss: 6.0458
Epoch 11/

J48 Decision Tree Regressor model

In [155]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Create a J48 Decision Tree Regressor model
regressor = DecisionTreeRegressor()

# Fit the model to the training data
regressor.fit(x_train, y_train)

# Make predictions on the test data
y_pred = regressor.predict(x_test)

# Calculate the Mean Squared Error (MSE) to evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 46.073495370370345


MLP Regressor model

In [156]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
regressor = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 4510.891569395584


User-Item Collaborative Filtering

In [157]:
# from surprise import SVD
# from surprise import Dataset
# from surprise import accuracy
# from surprise.model_selection import train_test_split

# # Create a SVD model
# model = SVD()

# # Load your data into the Surprise dataset
# data = Dataset.load_builtin('ml-100k')

# # Split the data into training and test sets
# trainset, testset = train_test_split(data, test_size=0.2)

# # Train the model on the training set
# model.fit(trainset)

# # Make predictions on the test set
# predictions = model.test(testset)

# # Calculate the Mean Squared Error (MSE) to evaluate the model
# mse = accuracy.mse(predictions)
# print("Mean Squared Error:", mse)


CART(Classification and regression tree)

In [158]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import mean_squared_error

# # Create a Decision Tree Classifier model
# model = DecisionTreeClassifier()

# # Train the model on the training data
# model.fit(x_train,y_train)  # Here, x_train represents user-item interaction data, and y_train represents target labels (e.g., ratings or preferences)

# # Make predictions on the test data
# y_pred = model.predict(x_test)  # x_test represents test data, and y_pred will be user-item interaction predictions

# # Calculate the Mean Squared Error (MSE) to evaluate the model
# mse = mean_squared_error(y_test, y_pred)  # y_test is the actual user-item interactions in the test set
# print("Mean Squared Error:", mse)
